In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("/home/cat/code/widefield/") # Adds higher directory to python modules path.
sys.path.append("/home/cat/code/manifolds/") # Adds higher directory to python modules path.


import numpy, scipy.optimize

import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm
import pandas as pd
import scipy

import os
import time
import warnings
warnings.filterwarnings("ignore")

from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax


from locanmf import LocaNMF, postprocess
from locanmf import analysis_fig4 


import umap


Autosaving every 180 seconds


In [2]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means



def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                          13,2,
                          12,3,
                          11,4,
                          10,5,
                          9,6,
                          8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        #print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    # print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/home/cat/code/widefield/locanmf/atlas_fixed_pixel.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    #print ("  # of ordered_names: ", ordered_names.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random
  
        
# #
def compute_variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)#[:,:900]
    var = np.array(var)#[:,:900]
    #print (means.shape, var.shape)

    return means, var


def plot_variance_locaNMF(var):
    colors = plt.cm.jet(np.linspace(0,1,var.shape[0]))

    scale1 = 1
    scale2 = .005
    linewidth = 2
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    #print ("t: ", t)
    fig=plt.figure(figsize=(10,10))
    for k in range(var.shape[0]):
        ax=plt.subplot(121)
        plt.xlim(-15,0)
        plt.xticks([])
        plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # 
        #plt.plot(t,temp,'--', c=colors[k],
        plt.plot(t,temp, c=colors[k],
                linewidth=linewidth)

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c=colors[k],alpha=.5)

    if False:
        plt.savefig('/home/cat/variance.png',dpi=300)
        plt.close()
    else:
        plt.show()
  

def load_variances(animal_id):
    session_name = 'all'
    root_dir = '/media/cat/4TBSSD/yuki/'

    # 
    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("# of sessions: ", loca.sessions.shape)

    # 
    all_means = []
    saved_names = []
    all_means_random = []
    good_sessions = []
    all_vars = []
    all_vars_random = []
    all_n_trials = []
    for session in tqdm(loca.sessions):

        # load data
        fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                     session + '_locanmf.npz')
        
        # 
        atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

        # 
        if atlas is not None:
            means, var = compute_variance_locaNMF(locaNMF_temporal)
            all_n_trials.append(locaNMF_temporal.shape[0])
            all_means.append(means)
            all_vars.append(var)
            # 
            means_r, var_r = compute_variance_locaNMF(random)
            all_means_random.append(means_r)
            all_vars_random.append(var_r) 

            saved_names = names
            good_sessions.append(session)

    #print ("# good sessions: ", len(good_sessions))
    
    return all_means, all_vars, saved_names, all_n_trials

############################################################
def compute_and_plot_variance_peaks(var, 
                                   means,
                                   n_trials,
                                   saved_names,
                                   session_id,
                                   plotting=False,
                                   smoothing=False):
    
    # 
    colors = plt.cm.jet(np.linspace(0,1,var.shape[0]))

    scale1 = 1.5E-4
    scale2 = 1E-6
    
    linewidth = 2

    # 
    if plotting:
        fig = plt.figure(figsize=(10,10))
        ax1 = plt.subplot(3,1,1)
        ax2 = plt.subplot(3,1,2)
        plt.xticks(np.arange(16), saved_names, 
                   rotation='vertical',
                   fontsize=14)
        #
        ax3 = plt.subplot(3,1,3)
        
        ax1.set_xlim(-30,0)
        
    # 
    ratios = []
    times = []
    # loop over all ROIs
    for k in range(var.shape[0]):

        # 
        temp = var[k]
        
        if True:
            temp3 = temp                    
        else:
            temp2 = means[k]

            temp2 += 100

            temp3 = temp/temp2
            temp3 = temp3*scale1+k*scale2

            # 
        if smoothing:
            window_length = 31
            polyorder = 3
            temp3 = scipy.signal.savgol_filter(temp3, 
                                               window_length, polyorder)
                                       
        # 
        if plotting:
            t = np.arange(temp3.shape[0])/30-30

            ax1.plot(t,temp3*scale1+k*scale2, c=colors[k],
                     linewidth=linewidth)

            ax1.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.5)
        
        # compute mean and plot it
        trace_control = np.mean(temp3[:temp3.shape[0]//2])
        if plotting:
            ax1.plot([-30,0], [trace_control*scale1+k*scale2,
                           trace_control*scale1+k*scale2],'--',c=colors[k], alpha=.5)

        
        #
        min_val = np.min(temp3[temp3.shape[0]//2:])
        ratio = min_val/trace_control
        
        # NOTE WE ONLY CONSIDER last 5 sec of data and exlcuding last 500ms
        #arg_min = np.argmin(temp3[temp3.shape[0]//2:-15])
        # print ('temp3 shape: ', temp3.shape)
        #arg_min = np.argmin(temp3[750:900-15])
        
        
        
        #arg_min = np.argmin(temp3[750:900-15])
        arg_min = np.argmin(temp3[450:900])
        arg_min = arg_min/30.-15

        
        
        # 
        if plotting:
            arg_min_int = int(arg_min*30+15)
            
            ax1.scatter(t[arg_min_int+temp3.shape[0]//2],
                        temp3[arg_min_int+temp3.shape[0]//2]*scale1+k*scale2, 
                        #color=colors[k],
                        color='black',
                        s=100)
        
        ratios.append(ratio)
        times.append(arg_min)

        if plotting:
            ax2.scatter(k, ratio,
                        s=300,
                       #color=colors[k]
                        color='black'
                       )
            
            ax2.set_ylim(0,1.0)
            ax2.set_xlim(-0.5,15.5)

            if k==0:
                for p in range(0,100,25):
                    ax2.plot([-0.5,16], [p/100.,p/100.],'--', c='black',alpha=.5)
                
            ax3.scatter(k, arg_min,
                        s=300,
                        #color=colors[k]
                        color='black',
                        )
            
            ax3.set_ylim(-5.0,0)
            ax3.set_xlim(-0.5,15.5)

            if k==0:
                for p in range(-5,1,1):
                    ax3.plot([-0.5,16],[p,p],'--', c='black',alpha=.5)
            #ax3.set_xticks([])
            #ax3.set_yticks([])
        
        # 
        if False:
            print (k, " area ", saved_names[k], 
               " ratio change: ", round(ratio,2),
               " time: ", round(arg_min,2))
        
        #return 
# 
    if plotting:
        plt.suptitle(animal_id+"  session: "+str(session_id) + 
             "    n trials: "+str(n_trials))

        if True:
            fig.savefig('/home/cat/variance.svg',dpi=300)
            #fig2.savefig('/home/cat/ratios.svg',dpi=300)
            #fig3.savefig('/home/cat/times.svg',dpi=300)
            plt.close('all')
        else:
            plt.show()

    return ratios, times

# 
#    
def plot_histograms_ratios(all_ratios, all_times, color,
                          min_ratio_threshold = 0.15):
    min_ratio_threshold = 0.15
    bin_width = .25
    bins = np.arange(-5,0.5,bin_width)
    good_vals = []

    plotting = False
    for k in range(all_ratios.shape[0]):
        idx = np.where(all_ratios[k]<min_ratio_threshold)[0]

        good_times = all_times[k][idx]
        good_vals.append(good_times)

        if plotting:
            y = np.histogram(good_times,bins = bins)

            plt.plot(y[1][1:],y[0],
                    c = colors[k])

    good_vals = np.hstack(good_vals)
    y = np.histogram(good_vals,bins = bins)

    plt.plot(y[1][1:],y[0],
            c = color)
    

    
def plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                saved_names, plotting,
                                session_id = None,
                                smoothing = False):
    
    
    if session_id is None:
        session_ids = np.arange(len(all_vars))
        if plotting==True:
            print (" CAN't plot all sesssions, exiting")
            return
    else:
        session_ids = [session_id]

    
    all_ratios = []
    all_times = []
    for session_id in session_ids:
        # session_id = 3
        # print (session_id, "n_trials: ", all_n_trials[session_id])

        # 
        ratios, times = compute_and_plot_variance_peaks(all_vars[session_id], 
                                               all_means[session_id],
                                               all_n_trials[session_id],
                                               saved_names,
                                               session_id,
                                               plotting,
                                               smoothing)
        all_ratios.append(ratios)
        all_times.append(times)

        # if plotting input sessoin_id manually and use break
        # break

    all_ratios = np.array(all_ratios)
    all_times = np.array(all_times)
    print (all_times.shape)
    print ("DONE")
    
    return all_ratios, all_times


def plot_box_plots_times(all_ratios, all_times, 
                         min_ratio_threshold=0.15):
    
    
    codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
    #codes = ['limb, layer 1 - right', 'limb, layer 1 - left']
    clrs_local = ['black','blue','red','magenta', 'pink','brown']
    
    # 
    bin_width = .25
    bins = np.arange(-5,0.5,bin_width)

    # 
    edts = []
    for a in range(len(all_ratios)):
        good_vals = []
        for k in range(all_ratios[a].shape[0]):
            idx = np.where(all_ratios[a][k]<min_ratio_threshold)[0]

            if idx.shape[0]>0:
                good_times = all_times[a][k][idx]
                good_vals.append(good_times)

            # 
        good_vals = np.hstack(good_vals)

        edts.append(good_vals)
        
    print ("M2: ", edts[1])
    
    # 
    my_dict = dict(M1 = edts[0], 
                   M2 = edts[1],
                   M3 = edts[2],
                   M4 = edts[3],
                   M5 = edts[4],                      
                   M6 = edts[5]
                     )

    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    # 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)


    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
#         plt.plot(x, y, 
#                  mfc = clrs_local[i], 
#                  mec='k', 
#                  ms=7, 
#                  marker="o", 
#                  linestyle="None",
#                     )
        #
        colors = plt.cm.viridis(np.linspace(0,1,len(edts[i])))
        x = np.random.normal(i+1, 0.04, len(edts[i]))
        print (i,d, ' y shape: ', y.shape)
        plt.scatter(x, edts[i], 
                   #c=clrs_local[i],
                   c=colors,
                   edgecolor='black',
                   s=200,
                   #alpha=np.linspace(.2, 1.0, x.shape[0])
                   alpha=.5
                   )

        
        
        res = scipy.stats.normaltest(edts[i])
        print ("i: ", i, '  res: ', res)
        
#     pvals = [0.05,0.01,0.001,0.0001,0.00001]
#     patches = []
#     if len(codes)!=2:
#         for p in [0,2,4]:
#             res = stats.ks_2samp(edts_saved[p], edts_saved[3])
#             label_ = ''
#             for k in range(len(pvals)):
#                 if res[1]<pvals[k]:
#                     label_ = label_ + "*"
#                 else:
#                     break

#                 patches.append(mpatches.Patch(color='blue', label=label_))





def load_locaNMF_temporal(animal_id, 
                          session_name, 
                          root_dir,
                          session_id):

    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    #print ("sessions: ", loca.sessions.shape)
    #print ("selected session: ", loca.sessions[session_id])

    session = loca.sessions[session_id]

    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')


    atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random = load_locaNMF_data(fname_locaNMF)

    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random 



def fit_sin(tt, yy):
    
    '''Fit sin to the input time sequence, 
        and return fitting parameters 
        "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"
        
        
    '''
    
    # 
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  
        return A * numpy.sin(w*t + p) + c
    
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    
    # 
    return {"amp": A, 
            "omega": w, 
            "phase": p, 
            "offset": c, 
            "freq": f, 
            "period": 1./f, 
            "fitfunc": fitfunc, 
            "maxcov": numpy.max(pcov), 
            "rawres": (guess,popt,pcov)
           }


def plot_phases(session_id,
                animal_id,
                session_name,
                root_dir,
                random_flag,
                areas_to_plot_phases,
                start,
                end,
                show_area_id,
                codes,
                clrs_local,
                plotting):
    
    ''' function that fits phases to raw locanm components
    
    '''
    
    try:
        #print ("session_id: ", session_id)
        (atlas, 
         areas, 
         names, 
         locaNMF_temporal, 
         locaNMF_temporal_random) = load_locaNMF_temporal(animal_id, 
                                                          session_name, 
                                                          root_dir,
                                                          session_id)
    except:
       # print (" couldn't load file")
        return ([[],[],[],[],[]])
        
        #     for ctr, name in enumerate(names):

#         print (ctr, name)
        
    #print ("GOT HERE")
    if atlas is None:
        #print ("session is empty ")
        return ([[],[],[],[],[]])
    #
    if plotting:
        fig=plt.figure(figsize=(15,10))
        colors = plt.cm.viridis(np.linspace(0,1,len(locaNMF_temporal)))

    # loop over areas 
    t0_phases=[]

    for ctr_area, area_sel in enumerate(areas_to_plot_phases):
        #    
        t0_phases.append([])

        #
        areas_selected = []
        for k in range(len(names)):
            if area_sel in names[k]:
                areas_selected.append(k)

        # use random data instead
        if random_flag:
            locaNMF_temporal = locaNMF_temporal_random
            
        #
        areas_selected = np.array(areas_selected)
        locaNMF_temporal2 = locaNMF_temporal[:,areas_selected].squeeze()

        # 
        missed_fit = 0
        t = np.arange(locaNMF_temporal2.shape[1])/30-30
        curves=[]
        
       # print ("# trials: ", locaNMF_temporal2.shape[0])
        for k in range(locaNMF_temporal2.shape[0]):
            if ctr_area==show_area_id:
                if plotting:
                    ax=plt.subplot(3,1,1)
                    plt.plot(t,locaNMF_temporal2[k], 
                         c='black',
                         linewidth=3,
                         alpha=.1)

            # fit sinusoid to the single trial data
            tt = t[start:end]
            yy = locaNMF_temporal2[k][start:end]

            # 
            try:
                tt2 = np.arange(0, locaNMF_temporal2[k].shape[0],1)/30.-30
                res = fit_sin(tt, yy)
                curve = res["fitfunc"](tt2)

                #print ("CURVE: ", curve.shape)
                # if curve fit, extrapolate all the way to -30sec
                
                if plotting:
                    if ctr_area==show_area_id:
                        ax=plt.subplot(3,1,2)
                        curve2 = (curve-np.min(curve))/(np.max(curve)-np.min(curve))*2-1
                        curves.append(curve2)
                        plt.plot(tt2, 
                                 curve2,  
                                 linewidth=2,
                                 c=clrs_local[show_area_id],
                                 alpha=.5
                            )
                        plt.scatter(tt2[900], 
                                 curve2[900],  
                                 edgecolor='black',
                                 c=clrs_local[show_area_id],
                                 alpha=1
                            )

                # get peaks of curves
                phase = get_angle_from_sinusoid(curve, 900)
                    
                #
                t0_phases[ctr_area].append(phase)

            #
            except:
                t0_phases[ctr_area].append(np.nan)
                missed_fit+=1


        ###############################
        ######## PLOT MEANS ###########
        ###############################
        if ctr_area==show_area_id:
            #curves=np.array(curves)
            mean = np.mean(locaNMF_temporal2,axis=0)
            if plotting:
                ax=plt.subplot(3,1,1)
                plt.plot(t,mean, 
                         c='black',
                         linewidth=8,
                         alpha=1)

                # 
                plt.plot([-30,30],[0,0],'--',c='grey')
                plt.xlim(-15,1)
                plt.plot([0,0],[-10,10],'--',c='grey')
                plt.ylim(-0.15,0.20)


            # fit sinusoid to trial average
            try:
                tt = t[start:end]
                curves=np.array(curves)
                mean = np.mean(curves,axis=0)
                tt2 = np.arange(0,mean.shape[0],1)/30.-30
                yy = mean[start:end]

                res = fit_sin(tt, yy)
                #print( "Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )
                
                # PLOT REAL AVERAGE NOT FIT 
                
                if plotting:
                    mean = np.mean(curves,axis=0)
                    ax=plt.subplot(3,1,2)
                    #plt.plot(tt, yy, "-k", label="y", linewidth=2)
                    plt.plot(tt2, mean, linewidth=8,
                            c=clrs_local[show_area_id]
                            )

                    # 
                    plt.plot([-30,30],[0,0],'--',c='grey')
                    plt.plot([0,0],[-10,10],'--',c='grey')
                    plt.xlim(-15,1)
                    plt.ylim(-0.15,0.15)
                    plt.ylim(-1,1)
            except:
                pass
                

    ###############################################################
    ###############################################################
    ###############################################################

    my_dict = dict(restrosplenial = t0_phases[0],
                   barrel = t0_phases[1],
                   limb = t0_phases[2],
                   visual = t0_phases[3],
                   motor = t0_phases[4])

    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    # 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    if plotting:
        ax = fig.add_subplot(313)
        data.boxplot(showfliers=False,
                flierprops=flierprops)

        # 
        for i,d in enumerate(data):
            colors = plt.cm.viridis(np.linspace(0,1,len(t0_phases[i])))
            y = data[d]
            x = np.random.normal(i+1, 0.04, len(y))
            if False:
                plt.plot(x, y, 
                     mfc = 'black', 
                     mec='k', 
                     ms=7, 
                     marker="o", 
                     linestyle="None",
                        )
            #
            else:
                x = np.random.normal(i+1, 0.04, len(t0_phases[i]))
                #print (i,d, ' y shape: ', y.shape)
                plt.scatter(x, t0_phases[i], 
                           #c=clrs_local[i],
                           c=clrs_local[i],
                           edgecolor='black',
                           s=200,
                           #alpha=np.linspace(.2, 1.0, x.shape[0])
                           alpha=.2
                           )

        plt.ylim(0,2*np.pi)
        
    # print ("# of non-fit trias: ", missed_fit)
    
    return t0_phases


def plot_polar_plots(t0_phases):
    
    fig2=plt.figure(figsize=(15,5))

    # 
    for k in range(len(t0_phases)):
        ax = fig2.add_subplot(1,5,k+1, projection='polar')

        N = 16

        #
        conversion= 1

        # 
        bins = np.linspace(0,2*np.pi*conversion, N+1)
        phases1 = np.array(t0_phases[k])*conversion
        y = np.histogram(phases1, 
                        bins = bins
                        #bins = np.arange(0,360+45,45),
                        )
        #
        theta = y[0]#[:-1]
        theta = theta/np.max(theta)  # not necessary
        radii = y[1][1:]

        #
        width = (2*np.pi) / (N)
        ax.bar(radii-width/2., 
               theta, 
               width=width,
               color=clrs_local[k])
        
        # 
        ax.set_yticklabels([])
        #plt.title(codes[k])
        ax.yaxis.grid(False)
        
        #ax.set_xticks([])
        ax.xaxis.set_tick_params(labelsize=20)
        if True:
            ax.set_xticklabels([])

    plt.suptitle(animal_id)

    
    #if True:
    #    plt.savefig('/home/cat/polar.svg')
    #    plt.close()
    #else:
    #    plt.show()
    
    return fig2

def get_angle_from_sinusoid(curve, index):
    # COMPUTE PHASE ANGLE FROM AMPLITUDE AND GRADIENT OF SINUSOID
    # Index is the loatoin of the point at which the phase is to be computed
        
    # must normalize and recentre curve to go from -1 to +1 as a standardized sinusoid   
    curve = (curve-np.min(curve))/(np.max(curve)-np.min(curve))*2-1
            
    angle = np.arcsin(curve[index])
    if angle<0:
        angle+=np.pi

    #
    curve0 = curve[index]
    curve1 = curve[index+1]
    if curve0<0:
        if (curve1-curve0)>0:
            angle = np.pi+angle
        else:
            angle = 2*np.pi-angle
    else:
        if (curve1-curve0)<=0:
            angle = np.pi-angle
        else:
            angle = angle
            
    phase = angle
    
    return phase

In [3]:
##################################################################
############ GENERATE SCHMIDT ET AL 2016 BAR PLOTS ###############
##################################################################
import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * numpy.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

fig=plt.figure()

yy = [0.25, 0.29, 0.24, 0.17, 0.25,0.29]
x = np.arange(-1,5,1)*1.1
print (x)
tt2 = numpy.linspace(x[0], x[-1], 100)


res = fit_sin(x, yy)
print( "Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )

#plt.plot(x, yy, "-k", label="y", linewidth=2)
#plt.plot(x, yynoise, "ok", label="y with noise")
plt.plot(tt2, res["fitfunc"](tt2)*1.6-.24, 
         c='grey',
         linewidth=5)

plt.bar(x, yy,.5,
        color='black')
plt.ylim(0,0.43)
plt.xlim(-0.5,x[-2]+0.5)

if True:
    plt.savefig('/home/cat/schmidt.svg')
    plt.close()
else:
    plt.show()


[-1.1  0.   1.1  2.2  3.3  4.4]
Amplitude=-0.05798706841699871, Angular freq.=1.4279967566460996, phase=-1.4410938988193076, offset=0.23749999999546362, Max. Cov.=0.02132519169920449


In [80]:
######################################################
############ SCHMIDT BAR PLOTS FOR MICE DATA #########
######################################################

# NOTE NEED TO RUN NEXT CELL TO GET THE PHASE COMPUTATION
animal_ids = ['IA3','AQ2']
clrs_anims = ['red','blue']
codes = ['_Retrosplenial', '_barrel', '_limb', '_visual','_motor','']
clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']
feat = 4

# 
fig=plt.figure()
for ctra, animal_id in enumerate(animal_ids):
    data2 = np.vstack(np.load('/home/cat/all_phases_all_trials_'+animal_id+'.npy',
                  allow_pickle=True))

    # 
    data = []
    for k in range(len(data2)):
        data.append(data2[k][feat])

    if True:  # all sessions
        data = np.hstack(data)
    else:
        data = np.hstack(t0_phases[feat])  #single session


    #########################################
    yy = data/np.pi*180
    width = 90
    offset = 0
    bins = np.arange(offset,361+offset,width)
    y = np.histogram(yy, bins=bins)

    yy = y[0]
    yy = np.roll(yy,3)
    shift = 90
    plt.bar(y[1][:-1]+width//2+ctra*30, 
            yy/np.sum(yy),
            width//3,
            color=clrs_anims[ctra],
           alpha=.8)


    plt.xlim(y[1][0],y[1][-2]+width*1.5)


    #########################################
    if False and ctra==1:
        x = y[1][:-1]

        # 
        yyy = np.hstack((yy[-1],yy))
        yyy = np.hstack((yyy,yy[0]))
        
        #
        x = np.hstack((-90,x))
        x = np.hstack((x,360))
        res = fit_sin(x, yyy)

        # 
        tt2 = numpy.linspace(x[0], x[-1], 1000)
        
        if ctra==0:
            plt.plot(tt2, res["fitfunc"](tt2)*.003+0.03, 
                 color=clrs_anims[ctra],
                 linewidth=5)
        else:
            plt.plot(tt2, res["fitfunc"](tt2)*.0001+0.03, 
                 color=clrs_anims[ctra],
                 linewidth=5)
            

if True:
    plt.savefig('/home/cat/schmidt_mouse_'+animal_id+"_"+str(feat)+'.svg')
    plt.close()
else:
    plt.show()



In [18]:
##################################################################
############ PLOT ALL TRIALS SINGLE SESSION + SINE FIT ###########
##################################################################

# FIG 5A
animal_id = 'IJ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'

# 
areas_to_plot_phases = ['Retrosplenial area, dorsal part, layer 1 - left',
                        'Primary somatosensory area, barrel field, layer 1 - left',
                        'upper limb, layer 1 - left',
                        'Primary visual area, layer 1 - left',
                        'Primary motor area, Layer 1 - left',
                       ]

#
codes = ['_Retrosplenial', '_barrel', '_limb', '_visual','_motor','']
clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']

#    
show_area_id = 4

session_id = 38

#
random_flag = False

start = 900 - 5*30   # start at -10 sec
end = 900 +0*30             # end at 0 sec

#start = 0
#end = 1800
#     
plotting=True
t0_phases = plot_phases(session_id,
                        animal_id,
                       session_name,
                       root_dir,
                       random_flag,
                       areas_to_plot_phases,
                       start,
                       end,
                       show_area_id,
                       codes,
                       clrs_local,
                       plotting)

if plotting:
    if False:
        plt.savefig('/home/cat/single_trial_phases_and_fits.svg')
        plt.close()
    else:
        plt.show()

In [5]:
##################################################
######## PLOT POLAR PLOTS FROM SIN FITS ##########
##################################################
print (len(t0_phases))
# Fig 5A Polar plots
    
plot_polar_plots(t0_phases)
if False:
    plt.savefig('/home/cat/polar_single.svg')
    plt.close()
else:
    plt.show()

5


In [8]:
###########################################
######## ALL SESSION POLAR PLOTS ##########
###########################################

#FIG 5B
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
#animal_ids = ['IA1']
ctr_a = 0
for animal_id in animal_ids:
    session_name = 'all'
    root_dir = '/media/cat/4TBSSD/yuki/'

    # 
    areas_to_plot_phases = ['Retrosplenial area, dorsal part, layer 1 - left',
                            'Primary somatosensory area, barrel field, layer 1 - left',
                            'upper limb, layer 1 - left',
                            'Primary visual area, layer 1 - left',
                            'Primary motor area, Layer 1 - left',
                           ]

    #
    codes = ['_Retrosplenial', '_barrel', '_limb', '_visual','_motor','']
    clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']

    #    
    show_area_id = 2

    #
    random_flag = False

    start = 900 - 10*30   # start at -10 sec
    end = 900 +0*30    
    plotting=False

    # grab a safe # of sessions to cover the maximum # of session in the longest animal
    session_ids = np.arange(150)

    if True:
        import parmap

        t0_array = parmap.map(plot_phases, 
                              session_ids,
                              animal_id,
                              session_name,
                              root_dir,
                              random_flag,
                              areas_to_plot_phases,
                              start,
                              end,
                              show_area_id,
                              codes,
                              clrs_local,
                              plotting,
                              pm_processes=16,
                              pm_pbar=True)

    else:
        t0_array = []
        for session_id in tqdm(session_ids):
            #     
            try:
                t0_phases = plot_phases(session_id,
                                        animal_id,
                                        session_name,
                                        root_dir,
                                        random_flag,
                                        areas_to_plot_phases,
                                        start,
                                        end,
                                        show_area_id,
                                        codes,
                                        clrs_local,
                                        plotting)

                if t0_phases is not None:
                    t0_array.append(t0_phases)
            except:
                break

    #
    #print (" DONE... # of sessions: ", len(t0_array))
    
    np.save('/home/cat/all_phases_all_trials_'+animal_id+'.npy', t0_array)
    
    #####################################
    ############ MAKE PLOTS #############
    #####################################
    t0_phases = []
    for k in range(5):
        t0_phases.append([])

    #
    #fig=plt.figure(figsize=(15,3))
    for k in range(len(t0_array)):
        for p in range(len(t0_array[0])):
            t0_phases[p].append(t0_array[k][p])

    for k in range(5):
        t0_phases[k] = np.hstack(t0_phases[k])


    print (t0_phases)
    fig = plot_polar_plots(t0_phases)
    
    # 
#     from astropy.stats import rayleightest
#     from astropy import units as u
#     data = np.array([130, 90, 0, 145])*u.deg
#     temp = rayleightest(data)     
#     print ("rayleightest test data: ", temp)
    
    import pycircstat
    for k in range(len(t0_phases)):
        temp = t0_phases[k]
        idx = np.where(np.isnan(temp)==False)[0]
        temp = temp[idx]
    
        p, z = pycircstat.tests.rayleigh(temp)
        print (animal_id, codes[k], "rayleightest test : ", p,z)
        


    #fig.tight_layout()
    #fig.tight_layout(pad=30.0)

    if True:
        plt.savefig('/home/cat/all_polar_'+animal_id+str(random_flag)+'.svg')
        plt.close()
    else:
        plt.show()
    ctr_a+=1
    

100%|██████████| 150/150 [00:02<00:00, 61.84it/s]


[array([2.27789058, 0.5927641 ,        nan, ..., 3.66128358, 2.50432948,
       1.04229803]), array([1.21191534, 5.96254419, 6.02725815, ..., 1.88328921, 2.59309501,
       2.76401776]), array([1.27230637, 6.15102352, 3.90228209, ..., 3.32406051, 2.57244533,
       2.89734659]), array([0.66272043, 5.62473237, 5.37073213, ..., 3.09997485, 2.94687004,
       3.21246873]), array([2.74864947, 6.17856544,        nan, ..., 3.20531483, 2.67065725,
       3.27383345])]
IA1 _Retrosplenial rayleightest test :  9.977474918425266e-58 127.9594657658012
IA1 _barrel rayleightest test :  9.323951347968332e-58 128.0314333676305
IA1 _limb rayleightest test :  7.978672185578156e-149 318.63907311878324
IA1 _visual rayleightest test :  3.274734845603135e-30 67.02638005170321
IA1 _motor rayleightest test :  8.248488540808741e-95 207.69368999503422


100%|██████████| 150/150 [00:06<00:00, 24.31it/s]


[array([5.89961238, 6.04543985, 5.9209397 , ..., 0.10430285, 2.67039156,
       0.72169519]), array([6.15177029, 5.68469674, 5.95868805, ..., 1.61523585, 5.36169439,
       5.95296047]), array([0.27966924, 5.71053723, 0.14076198, ..., 5.88983297, 5.73513399,
       0.5403343 ]), array([5.73051878, 5.81633877, 5.70298323, ..., 0.91687618, 0.95180758,
       0.05719796]), array([0.30182002, 5.78440807, 0.23016239, ..., 6.20194288, 5.77344682,
       0.41787992])]
IA2 _Retrosplenial rayleightest test :  1.5268125113111022e-25 56.62860949988555
IA2 _barrel rayleightest test :  1.2990034906589378e-40 90.49883204831893
IA2 _limb rayleightest test :  7.049167115392553e-46 102.27687189708492
IA2 _visual rayleightest test :  1.6970178465745354e-08 17.845196243562985
IA2 _motor rayleightest test :  1.687326823524283e-41 92.50661123305903


100%|██████████| 150/150 [00:06<00:00, 22.04it/s]


...

[array([2.2134178 , 4.87048956, 0.72566161, ..., 1.23922299, 0.79783221,
       2.45807019]), array([3.4500694 , 1.07886926, 2.96759066, ..., 2.13824961, 1.62835117,
       2.4585607 ]), array([3.3159781 , 6.2634927 , 1.44599254, ..., 1.10891544, 1.1728866 ,
       0.56619469]), array([1.88641349, 0.0083273 , 6.00255341, ..., 1.25672619, 1.1490101 ,
       1.74848153]), array([3.13998968, 0.1387013 , 1.20098638, ..., 1.18511188, 1.28499024,
       0.75340475])]
IJ1 _Retrosplenial rayleightest test :  2.9442590316949963e-80 177.97964640618216
IJ1 _barrel rayleightest test :  4.169992814234157e-165 356.3293334589796
IJ1 _limb rayleightest test :  3.637825220843066e-222 469.671261829425
IJ1 _visual rayleightest test :  1.6871652665774338e-53 119.25327918838609
IJ1 _motor rayleightest test :  1.901750274949236e-247 518.3745648808074


100%|██████████| 150/150 [00:04<00:00, 37.44it/s]


[array([1.01126159,        nan, 0.44295353, ..., 6.21311637,        nan,
       3.28257523]), array([1.10531098, 2.63274871, 1.84430062, ..., 2.95981782, 0.70559344,
       2.6315838 ]), array([       nan, 2.04795362, 0.08945689, ..., 1.36105564, 3.09137892,
       0.83345088]), array([5.24132093, 2.16850912, 0.10635219, ..., 3.2853443 , 2.51519808,
       3.811628  ]), array([       nan, 2.06422095, 1.40359496, ..., 1.43491408, 3.95774362,
       4.36794717])]
IJ2 _Retrosplenial rayleightest test :  1.1374202211487204e-178 386.58000575361643
IJ2 _barrel rayleightest test :  5.796557543037383e-238 505.30684833307777
IJ2 _limb rayleightest test :  2e-323 666.8340153982277
IJ2 _visual rayleightest test :  7.425041394274856e-108 238.31029668151209
IJ2 _motor rayleightest test :  2.2834835522039e-310 642.8444696796009


100%|██████████| 150/150 [00:10<00:00, 13.91it/s]


[array([3.30921253, 1.89925198, 2.9684714 , ..., 0.96375109, 1.0008446 ,
       4.38668283]), array([4.14235496, 2.24288474, 0.80807773, ..., 0.67753039, 2.02798188,
       2.87725154]), array([0.65487634, 1.88374065, 3.34204492, ..., 0.77020662, 1.93241642,
       2.64093606]), array([3.55304624, 2.26434774, 3.1805201 , ..., 0.53293449, 5.69666975,
       3.63934075]), array([3.87941595, 2.2512912 , 3.11505535, ..., 0.86168238, 1.93726973,
       2.63202576])]
AQ2 _Retrosplenial rayleightest test :  0.0 1028.2737296319651
AQ2 _barrel rayleightest test :  0.0 1294.713526539123
AQ2 _limb rayleightest test :  0.0 1841.6939055977346
AQ2 _visual rayleightest test :  3.1041995207462954e-269 603.4992987560049
AQ2 _motor rayleightest test :  0.0 1727.156162606811


In [108]:
# NOTE NEED TO RUN NEXT CELL TO GET THE PHASE COMPUTATION

def get_vecs(data):
    phases = []
    for k in range(len(data)):
        temp = data[k]
        if len(temp[0])>0:
            #print (temp)
            try:
                phases.append(np.vstack(temp))
            except:
                pass

    vecs = []
    for s in range(len(phases)):
        #print (phases[s].shape)
        for p in range(phases[s].shape[1]):
            if np.any(np.isnan(phases[s][:,p]))==False:
                vecs.append(phases[s][:,p])

    vecs=np.array(vecs)    
    print ("Vecs: ", vecs.shape)
    
    return vecs

# 
def knn_triage(th, pca_wf):
    
    from scipy.spatial import cKDTree
    
    tree = cKDTree(pca_wf)
    dist, ind = tree.query(pca_wf, k=51)
    dist = np.sum(dist, 1)
    idx_keep1 = dist <= np.percentile(dist, th)
    
    id2 = np.where(idx_keep1==1)[0]
    
    return id2




def plot_corrs(corrs, x):
    vals = []
    for p in range(feats.shape[0]):
        for q in range(p+1,feats.shape[0],1):
            #if p==1 or p==3 or q==1 or q==3:
            #    continue
            #plt.scatter(p, corrs[p,q],
            #           c=clrs_local[q])
            #plt.scatter(x, corrs[p,q],
            #           c=clrs_local[x])
            vals.append(corrs[p,q])

    return vals
            
#######################################################
#######################################################
#######################################################
# 
root_fname = '/home/cat/all_phases_all_trials_'
clrs_local = ['black','blue','red','green','magenta', 'cyan']

feat_names = ['retrosplenial','barrel','limb','visual','motor']
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
#animal_ids = ['IA1']
#

fig=plt.figure()
cc=0
corrs_array =[]
for animal_id in animal_ids:
    ax=plt.subplot(2,3,cc+1)
    ctr=0
    data = np.load(root_fname+animal_id+".npy",allow_pickle=True)

    #feats = np.array([1])
    feats = np.arange(5)
    #feats = np.array([0,2,4])

# 
    #vecs = (get_vecs(data)/np.pi*180+90)%360
    vecs = get_vecs(data)/np.pi*180
    vecs = vecs[:,feats]
    
    #
    from scipy.stats import pearsonr

    # calculate Pearson's correlation
    corrs = np.zeros((feats.shape[0],feats.shape[0]))
    for p in range(feats.shape[0]):
        for q in range(feats.shape[0]):
            if p==q:
                corrs[p,q]=np.nan
                continue
            corr, _ = pearsonr(vecs[:,p],vecs[:,q])
            #print (p,q, corr)
            corrs[p,q]= corr

    corrs_array.append(corrs)
    #vecs = vecs[:, feats]
    #print ("vecs into triage: ", vecs.shape)
    
    tseries = []
    for f in range(feats.shape[0]):
        vecs1 = vecs[:,f]#[:,None]
        width = 20
        y = np.histogram(vecs1,bins=np.arange(0,360,width))
        plt.plot(y[1][:-1],y[0]/np.max(y[0]),
                c=clrs_local[ctr], label=feat_names[f])
    
        ctr+=1
                
    if animal_id =='IA1':
        plt.legend()
    plt.title(animal_id)
    plt.show()

    cc+=1

###################################################################
fig = plt.figure(figsize=(6,4))
v = []
for k in range(len(animal_ids)):
    #ax=plt.subplot(2,3,k+1)
    
    vals = plot_corrs(corrs_array[k], k)
    v.append(vals)
       
my_dict = dict(m1 = v[0], 
               m2 = v[1],
               m3 = v[2],
               m4 = v[3],
               m5 = v[4],
               m6 = v[5],   
                )

data = pd.DataFrame.from_dict(my_dict, orient='index')
data = data.transpose()

# 
flierprops = dict(marker='o', 
                  #markerfacecolor='g', 
                  #markersize=10000,
                  linestyle='none', 
                  markeredgecolor='r',)

#
data.boxplot(showfliers=False)


for i,d in enumerate(data):
    y = data[d]
    x = np.random.normal(i+1, 0.04, len(y))
#     plt.plot(x, y, 
#              mfc =clrs_local[i], 
#              mec='k', 
#              ms=7, 
#              marker="o", 
#              linestyle="None",

#             )

    plt.scatter(x, y, 
       c='black',
       edgecolor='black',
       s=200,
       alpha=.4)

        
        
        
#####################################################

for p in range(len(v)):
    for q in range(p+1,6,1):
        ks = scipy.stats.ks_2samp(v[p],v[q])
        print (p,q,ks)         
        
        
#
plt.savefig('/home/cat/phase_correlations.svg')
plt.close()


Vecs:  (1233, 5)
Vecs:  (1412, 5)
Vecs:  (2277, 5)
Vecs:  (1559, 5)
Vecs:  (1769, 5)
Vecs:  (6375, 5)
0 1 KstestResult(statistic=0.2, pvalue=0.9944575548290717)
0 2 KstestResult(statistic=0.2, pvalue=0.9944575548290717)
0 3 KstestResult(statistic=0.3, pvalue=0.7869297884777761)
0 4 KstestResult(statistic=0.5, pvalue=0.16782134274394334)
0 5 KstestResult(statistic=0.7, pvalue=0.012340600575894691)
1 2 KstestResult(statistic=0.2, pvalue=0.9944575548290717)
1 3 KstestResult(statistic=0.4, pvalue=0.41752365281777043)
1 4 KstestResult(statistic=0.3, pvalue=0.7869297884777761)
1 5 KstestResult(statistic=0.9, pvalue=0.00021650176448938054)
2 3 KstestResult(statistic=0.3, pvalue=0.7869297884777761)
2 4 KstestResult(statistic=0.4, pvalue=0.41752365281777043)
2 5 KstestResult(statistic=0.8, pvalue=0.002056766762649115)
3 4 KstestResult(statistic=0.6, pvalue=0.05244755244755244)
3 5 KstestResult(statistic=0.9, pvalue=0.00021650176448938054)
4 5 KstestResult(statistic=0.7, pvalue=0.012340600575894

In [69]:
# 



In [30]:
# NOTE NEED TO RUN NEXT CELL TO GET THE PHASE COMPUTATION


data1 = np.vstack(np.load('/home/cat/all_phases_all_trials_IA3.npy',
              allow_pickle=True))

data2 = np.vstack(np.load('/home/cat/all_phases_all_trials_IA1.npy',
              allow_pickle=True))

#print (data)

def get_vecs(data):
    phases = []
    print ("# sessions: ", len(data))
    for k in range(len(data)):
        temp = data[k]
        if len(temp[0])>0:
            #print (temp)
            try:
                phases.append(np.vstack(temp))
            except:
                pass

    vecs = []
    for s in range(len(phases)):
        #print (phases[s].shape)
        for p in range(phases[s].shape[1]):
            if np.any(np.isnan(phases[s][:,p]))==False:
                vecs.append(phases[s][:,p])

    vecs=np.array(vecs)    
    print ("Vecs: ", vecs.shape)
    
    return vecs

vecs1 = get_vecs(data1)
vecs2 = get_vecs(data2)
vecs = np.vstack((vecs1, vecs2))
print (vecs1.shape, vecs2.shape, vecs.shape)    
    
    
from sklearn.decomposition import PCA

X_lever_pulls = vecs.copy()

# X_random = X_lever_pulls.copy().flatten()
# idx = np.random.choice(np.arange(X_random.shape[0]),
#                        X_random.shape[0],
#                        replace=False)
# X_random = X_random[idx].reshape(-1,5)
X_random = np.random.random((X_lever_pulls.shape[0]*10,
                             X_lever_pulls.shape[1]),
                            )*2*np.pi

#X_random = np.zeros(X)
print (X_random.shape)
print (X_random)
X_all = np.vstack((X_lever_pulls, X_random))

X_all = (X_lever_pulls)

##########################
if True:
    pca = PCA(n_components=5)
    pca.fit(X_all)
    
    from mpl_toolkits.mplot3d import Axes3D

    # 
    X_pca = pca.transform(X_all)

else: 
    import umap.umap_ as umap

    def run_UMAP(data, 
                 n_neighbors=50,
                 min_dist=0.1,
                 n_components=3,
                 metric='euclidean'):

        fit = umap.UMAP(
            n_neighbors=n_neighbors,
            min_dist=min_dist,
            n_components=n_components,
            metric=metric
        )

        u = fit.fit_transform(data)

        return u


    X_pca = run_UMAP(X_all)

    
print ("DONE...", X_pca.shape)

# sessions:  150
Vecs:  (2277, 5)
# sessions:  150
Vecs:  (1233, 5)
(2277, 5) (1233, 5) (3510, 5)
(35100, 5)
[[1.39585597 1.81475082 4.13988176 5.83648705 1.97970151]
 [6.0681404  0.65993783 3.74755023 4.20457456 3.72195353]
 [3.36295774 5.740311   4.09780588 0.03991259 2.69199289]
 ...
 [0.33218459 1.47451029 6.14579351 2.23031597 1.12540535]
 [4.17591915 2.23430683 0.24060242 3.98653185 1.98590518]
 [3.32058524 2.90778552 1.18019485 5.03101169 4.45479041]]
DONE... (3510, 5)


In [33]:

###############################################################
###############################################################
###############################################################
v = Visualize2()
v.keep_plot = True

#
title='velocity (m/s) '
cmap = 'Blues'
time = np.arange(vecs1.shape[0])

X_in = X_pca[:vecs1.shape[0]]
print ("XPCA: ", X_pca.shape)
print ("Xin : ", X_in.shape)

fig = v.plot_scatter_3d( 
                     X_in[:,:3], 
                     time,
                     title,
                     cmap,
                   1,1
                  )

cmap = 'Reds'
time = np.arange(vecs2.shape[0])
X_in = X_pca[vecs1.shape[0]:vecs1.shape[0]+vecs2.shape[0]]
fig = v.plot_scatter_3d( 
                   X_in[:,:3], 
                 time,
                 title,
                 cmap,
                   1,1,
    fig
                  )

# cmap = 'Greys'
# time = np.arange(vecs1.shape[0]+vecs2.shape[0])
# X_in = X_pca[vecs1.shape[0]+vecs2.shape[0]:]
# fig = v.plot_scatter_3d( 
#                    X_in[:,:3], 
#                  time,
#                  title,
#                  cmap,
#                    1,1,
#     fig
#                   )


fig.show()



XPCA:  (3510, 5)
Xin :  (2277, 5)
DATA:  (2277, 3)


Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

In [34]:

###############################################################
###############################################################
###############################################################
v = Visualize2()
v.keep_plot = True

#
title='velocity (m/s) '
cmap = 'Blues'
time = np.arange(vecs1.shape[0])

n_d = 3
X_in = X_pca[:vecs1.shape[0]]
print ("XPCA: ", X_pca.shape)
print ("Xin : ", X_in.shape)

fig = v.plot_scatter_3d( 
                     X_in[:,:n_d], 
                     time,
                     title,
                     cmap,
                   1,1,
    None,
    3
                  )

cmap = 'Reds'
time = np.arange(vecs2.shape[0])
X_in = X_pca[vecs1.shape[0]:vecs1.shape[0]+vecs2.shape[0]]
fig = v.plot_scatter_3d( 
                   X_in[:,:n_d], 
                 time,
                 title,
                 cmap,
                   1,1,
    fig,3
                  )

# cmap = 'Greys'
# time = np.arange(vecs1.shape[0]+vecs2.shape[0])
# X_in = X_pca[vecs1.shape[0]+vecs2.shape[0]:]
# fig = v.plot_scatter_3d( 
#                    X_in[:,:3], 
#                  time,
#                  title,
#                  cmap,
#                    1,1,
#     fig
#                   )


fig.show()


XPCA:  (3510, 5)
Xin :  (2277, 5)
DATA:  (2277, 3)
DATA:  (1233, 3)


In [24]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

class Visualize2():

    def __init__(self):
        self.keep_plot = True

    def plot_traces(self, traces, n=100):

        if self.keep_plot==False:
            plt.figure()

        t = np.arange(traces.shape[1] ) /self.sample_rate
        for k in trange(n):
            plt.plot(t ,traces[k ] + 1 *k)

        plt.ylabel("First 100 neurons")
        plt.xlabel("Time (sec)")
        # plt.yticks([])
        plt.xlim(t[0] ,t[-1])
        plt.show()

    def plot_raster(self, ax, rasters, time_stamps):



        ax1.imshow(bn,
                   aspect='auto', cmap='Greys',
                   interpolation='none')

        # ax1.xlabel("Imaging frame")
        ax1.set_ylabel("Neuron", fontsize=20)
        ax1.set_xticks([])


    def plot_scatter_3d(self,
                     data,
                     clrs=None,
                     title='',
                     cmap='viridis',
                     rows=1,
                     cols=1,
                     fig=None,
                     size=2,
                     cbar=True
                     ):

        #
        if clrs is None:
            clrs = np.arange(data.shape[0])

        #
        print ("DATA: ", data.shape)
        #
        if data.shape[1] == 2:

            if fig is None:
                fig = make_subplots(rows=1, cols=1,
                                specs=[[{'type': 'scene'}]])

                
            #fig = make_subplots(rows=1, cols=1)
            #
            df = pd.DataFrame(data[:, :2],
                              columns=["DIM1", 'DIM2'])
            print (df)
            fig.scatter(
                df, x='DIM1', y='DIM2',
                color=clrs,
                width=1200,
                height=750
            )

            fig.layout.coloraxis.colorbar.title = title
            if cbar:
                fig.layout.coloraxis.colorscale = cmap
                fig.layout.coloraxis.colorbar.title = title
            fig.layout.coloraxis.colorscale = cmap
            #fig.show()


        #
        else:
            if fig is None:
                fig = make_subplots(rows=1, cols=1,
                                specs=[[{'type': 'scene'}]])


            if cbar:

                fig.add_traces([go.Scatter3d(
                    x=data[:, 0],
                    y=data[:, 1],
                    z=data[:, 2],
                    mode='markers',
                    marker=dict(size=size,
                                color=clrs,
                                colorscale=cmap,
                                opacity=1,
                                colorbar=dict(thickness=25,
                                              title=title,
                                              #textangle=-90,
                                              outlinewidth=0
                                              )

                                )
                ),

                ],

                    rows=rows,
                    cols=cols)

               # fig.
                fig.layout.coloraxis.colorbar.title = title
                #fig.update_layout(coloraxis_colorbar_angle=-90)
            else:
                fig.add_traces([go.Scatter3d(
                    x=data[:, 0],
                    y=data[:, 1],
                    z=data[:, 2],
                    mode='markers',
                    marker=dict(size=size,
                                color=clrs,
                                colorscale=cmap,
                                opacity=1
                                )
                ),

                ],

                    rows=rows,
                    cols=cols)

                #fig.layout.coloraxis.colorbar.hide_colorbar()
                fig.update_layout(coloraxis_showscale=False)

            #fig.layout.coloraxis.colorscale = cmap
            fig.update_layout(height=800, width=1400)

            #fig.layout.coloraxis.colorscale = cmap
            #fig.show()

        return fig

In [109]:
#############################################
######### LONGITUDINAL PHASE PLOTS ##########  NOT USED
#############################################

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']

animal_id = animal_ids[2]
fname = '/home/cat/all_phases_all_trials_'+animal_id+'.npy'
print (fname)
data = np.load(fname,allow_pickle=True)

print (len(data), len(data[0]))
from tqdm import trange

res = []
for k in range(5):
    res.append([])
for k in trange(len(data)):
    temp =  np.array(data[k])
    
    if len(temp[0])==0:
        continue
        
    for p in range(temp.shape[0]):
        
        N = 16

        # 
        bins = np.linspace(0,2*np.pi, N+1)
        phases1 = np.array(temp[p])
        y = np.histogram(phases1, 
                         bins = bins
                        )
        #
        theta = y[0] *180/np.pi
        #theta = theta/np.max(theta)  # not necessary
        radii = y[1][1:]*180/np.pi

        idx = np.argmax(theta)
        
        res[p].append(radii[idx])

        #plt.scatter(k, [radii[idx]],
        #           color=clrs_local[p])

        
for k in range(len(res)):
    plt.plot(res[k],
            color=clrs_local[k])
        
plt.ylim(0,360)
plt.xlim(left=0)
plt.show()

100%|██████████| 150/150 [00:00<00:00, 3961.40it/s]

/home/cat/all_phases_all_trials_IA3.npy
150 5


In [48]:
# ###########################################################
# ########### LOAD RAW DATA OPTIONAL - FIG 4A ONLY ##########
# ###########################################################

# spatial_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy'
# temporal_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
    
# raw_means = load_raw_data(spatial_fname, temporal_fname)

# Plot locaNMF vs. raw
# plot_locanmf_vs_raw(locaNMF_temporal, raw_means)

In [174]:
######################################################
###### LOAD LOCANMF DATA AND CORRECT ORDER MAPS ######
######################################################

# 
animal_id = 'IA3'
all_means, all_vars, saved_names, all_n_trials = load_variances(animal_id)

  5%|▍         | 2/44 [00:00<00:02, 17.86it/s]

# of sessions:  (44,)


100%|██████████| 44/44 [00:02<00:00, 15.90it/s]


In [175]:
###################################################
##### PLOT LONGITUDINAL MEANS AND VARIANCES #######
###################################################

# Fig 5B
#for k in range(len(saved_names)):
#    print (k, saved_names[k])

# 
print (len(all_vars))
session_ids = np.arange(len(all_vars))

# colors
roi_ids = [1,9,11,15]

print ("plotitng: ", saved_names[roi_id])

colors = plt.cm.viridis(np.linspace(0,1,len(all_vars)))


plot_vars = True
fig = plt.figure(figsize=(20,12))
for ctr, roi_id in enumerate(roi_ids):
    ax = plt.subplot(2,2,ctr+1)
    all_ = []
    for ctr_sess, session_id in enumerate(session_ids):
        if plot_vars:
            temp = all_vars[session_id][roi_id].copy()
        else:
            temp = all_means[session_id][roi_id].copy()
        t = np.arange(temp.shape[0])/30.-30

        # 
        if False:
            temp = savgol_filter(temp, 11, 3)

        elif False:
            fs = 30
            fc = 1  # Cut-off frequency of the filter


            w = fc / (fs / 2) # Normalize the frequency
            b, a = signal.butter(5, w, 'low')
            temp = signal.filtfilt(b, a, temp)

        #mean = temp.mean(1)
        temp[:400]=0
        temp[1350:]=0
        plt.plot(t,temp, color=colors[ctr_sess],alpha=.5,
                linewidth=3)

        all_.append(temp)

    a = np.array(all_)
    plt.plot(t, np.mean(a,axis=0),c='red',
            linewidth=5)
    plt.xlim(-15,5)
    
    # 
    if plot_vars:
        plt.ylim(0,.006)
        #plt.ylim(bottom=0)
        plt.plot([-30,30],[0,0],'--',c='grey')
        plt.plot([0,0],[-.2,.2],'--',c='grey')
    else:
        plt.ylim(-.075,.15)
        plt.plot([-30,30],[0,0],'--',c='grey')
        plt.plot([0,0],[-.2,.2],'--',c='grey')
    plt.title(saved_names[roi_id])
    
plt.suptitle(animal_id)
if True:
    plt.savefig('/home/cat/variance_analysis_var'+str(plot_vars)+'.svg')
    plt.close()
else:
    plt.show()

41
plotitng:  Retrosplenial area, dorsal part, layer 1 - left


In [179]:


a = np.array([[0,1]])
plt.figure(figsize=(1.5, 9))
img = plt.imshow(a, cmap="viridis")
plt.gca().set_visible(False)
cax = plt.axes([0.1, 0.2, 0.8, 0.6])
plt.colorbar(orientation="vertical", cax=cax)
plt.savefig("/home/cat/colorbar.svg")


In [165]:
###############################
###### VARIANCE ANALYSIS ######
###############################

# FIG 6 H variance curves + times + ratios

plotting = True  # note you need to run plotting=True for a single sessoin not all!!!
session_id = -1

saved_names = [
'M1-R',
'M1-L',
'M2-R',
'M2-L',
'SomB-R',
'SomB-L',
'SomH-R',
'SomH-L',
'SomF-R',
'SomF-L',
'V1-R',
'V1-L',
'RL-R',
'RL-L',
'RD-R',
'RD-L'    
]

smoothing = False
all_ratios, all_times = plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                                    saved_names, plotting,
                                                    session_id,
                                                    smoothing)

(1, 16)
DONE


In [172]:
#############################################
###### ALL ANIMAL LOWEST VARIANCE TIME ######
#############################################

animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']    
#animal_ids = ['IA3']
session_id = None

# 
ratios = []
times= []
smoothing = True
for ctr,animal_id in enumerate(animal_ids):
    all_means, all_vars, saved_names, all_n_trials= load_variances(animal_id)

    plotting = False  # note you need to run plotting=True for a single sessoin not all!!!
    all_ratios, all_times = plot_variance_peaks_3_plots(all_vars, all_means, all_n_trials,
                                                        saved_names, 
                                                        plotting,
                                                        session_id,
                                                        smoothing)
    ratios.append(all_ratios)
    times.append(all_times)
    

fig = plt.figure(figsize=(10,10))
min_ratio_threshold = 0.25
plot_box_plots_times(ratios, times, 
                     min_ratio_threshold)
plt.ylim(-15,0)

if True:
    plt.savefig('/home/cat/min_var_times.svg')
    plt.close()
else:
    plt.show()



  4%|▍         | 3/71 [00:00<00:02, 27.62it/s]

# of sessions:  (71,)


  9%|▉         | 4/44 [00:00<00:01, 26.42it/s]

(53, 16)
DONE
# of sessions:  (44,)


  5%|▍         | 2/44 [00:00<00:02, 18.35it/s]

...

(40, 16)
DONE
# of sessions:  (44,)


 14%|█▎        | 15/110 [00:00<00:00, 130.33it/s]

(39, 16)
DONE
# of sessions:  (110,)


100%|██████████| 110/110 [00:07<00:00, 14.33it/s]


(77, 16)
DONE
M2:  [ -0.03333333  -0.03333333  -0.2         -0.23333333  -0.16666667
  -0.13333333  -0.13333333  -0.13333333 -11.86666667 -11.86666667
 -10.7        -10.7         -0.03333333  -0.03333333  -0.03333333
  -0.03333333  -0.03333333  -0.03333333  -0.03333333  -3.96666667
  -0.03333333  -0.03333333  -0.03333333  -0.03333333 -12.6
  -1.23333333  -1.26666667  -1.1         -1.06666667  -0.03333333
  -2.13333333  -0.03333333  -0.03333333  -1.96666667  -1.56666667
  -3.86666667  -0.2         -0.03333333  -0.03333333  -0.03333333
  -0.03333333  -0.03333333  -7.86666667  -7.83333333]
0 M1  y shape:  (258,)
i:  0   res:  NormaltestResult(statistic=65.04669956359484, pvalue=7.503927972521363e-15)
1 M2  y shape:  (258,)
i:  1   res:  NormaltestResult(statistic=21.733955008667895, pvalue=1.9077946866075673e-05)
2 M3  y shape:  (258,)
i:  2   res:  NormaltestResult(statistic=1.8937663104486255, pvalue=0.3879483156868674)
3 M4  y shape:  (258,)
i:  3   res:  NormaltestResult(statistic=238

In [37]:

data = np.load('/media/cat/4TBSSD/yuki/IA3/tif_files/IA3am_Mar15_30Hz/IA3am_Mar15_30Hz_locanmf.npz',
              allow_pickle=True)

A_reshape = data['A_reshape']
temp = data['temporal_trial']
names = data['areanames_area']

print (temp.shape)
mean = temp.mean(0)


fig=plt.figure()
for i in range(A_reshape.shape[2]):
    plt.subplot(4,4,i+1)
    plt.imshow(A_reshape[:,:,i])
    plt.xticks([])
    plt.yticks([])
    plt.title(names[i],fontsize=6)
plt.tight_layout(h_pad=0.5,w_pad=0.5)
plt.show()




(34, 16, 1801)


In [62]:
############################################


def get_sessions(main_dir,
                 animal_id,
                 session_id):
     # load ordered sessions from file
    sessions = np.load(os.path.join(main_dir,
                                         animal_id,
                                         'tif_files.npy'))
    # grab session names from saved .npy files
    data = []
    for k in range(len(sessions)):
        data.append(os.path.split(sessions[k])[1].replace('.tif',''))
    sessions = data

    #
    if session_id != 'all':
        final_session = []
        for k in range(len(sessions)):
            if session_id in sessions[k]:
                final_session = [sessions[k]]
                break
        sessions = final_session

    # fix binary string files issues; remove 'b and ' from file names
    for k in range(len(sessions)):
        sessions[k] = str(sessions[k]).replace("'b",'').replace("'","")
        if sessions[k][0]=='b':
            sessions[k] = sessions[k][1:]

    sessions = np.array(sessions)

    return sessions

main_dir = '/media/cat/4TBSSD/yuki/'
session_id = 'all'


    
animal_ids = ['AQ2']
fig =plt.figure(figsize=(10,10))
ax=plt.subplot()
xmax = 21
for animal_id in animal_ids: 
    sessions = get_sessions(main_dir,
                             animal_id,
                             session_id)
    # 
    colors = plt.cm.plasma(np.linspace(0,1,len(sessions)))
    for ctr,session in enumerate(sessions):
        #print ("session: ", session)

        # 
        fname_var = os.path.join(main_dir,animal_id,'tif_files',session,
                                             session+ '_whole_stack_trial_ROItimeCourses_15sec_pca_var_explained.npy')

        if os.path.exists(fname_var)==True:
        
            var = np.load(fname_var)

            var = np.cumsum(var)
            x=np.arange(var.shape[0])+1
            plt.plot(x[:xmax], 
                     var[:xmax],
                    color=colors[ctr],
                    linewidth=2,
                    alpha=.9)
            
            
            
        
#
plt.plot([0,xmax],[0.95,0.95],
        '--',c='black')
#plt.semilogy()
plt.ylim(.6,1)
plt.xlim(0,xmax-1)

cmap = plt.cm.plasma  # define the colormap

cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
cb = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap, spacing='proportional')
cb.set_label('gdp_md_est')




if True:
    plt.savefig('/home/cat/variance.svg', dpi=1200)
    plt.close()
else:
    plt.show()
        
    
    
    

In [28]:
d = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
print (d.shape)

(106, 6, 1801)


In [38]:
data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components_spatial.npy')
print (data.shape)

mask = np.int32(np.loadtxt('/media/cat/4TBSSD/yuki/IJ1/genericmask.txt'))

mask1 = np.ones((128,128),'float32')

for k in range(mask.shape[0]):
    mask1[mask[k][0],
          mask[k][1]] = np.nan
    
print (mask1.shape)

(30, 16384)
(128, 128)


In [41]:
for k in range(6):
    ax=plt.subplot(2,3,k+1)
    temp = data[k].reshape(128,128)*mask1
    plt.imshow(temp)
    
plt.savefig('/home/cat/pcas.svg',dpi=1200)
plt.close()

In [406]:
N, amp, omega, phase, offset, noise = 500, 5., 2., 0, 1.7, 3

angles_fitted = []
angles_truth = np.linspace(0,2*np.pi,100)
for phase in angles_truth:
#phase = 3*np.pi/2 +.7 #+1.0

    if phase>(2*np.pi):
        phase-=2*np.pi
    #
    tt = numpy.linspace(0, 10, N)-5
    tt2 = numpy.linspace(0, 10, 10*N)-5

    # 
    yy = amp*numpy.sin(omega*tt + phase) + offset
    yynoise = yy # + noise*(numpy.random.random(len(tt))-0.5)

    res = fit_sin(tt, yynoise)
   # print( "Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )

    #print ("phase in radians: ", res['phase']/np.pi)
    if False:
        fig=plt.figure()
        plt.plot(tt, yy, "-k", label="y", linewidth=2)
        plt.plot(tt, yynoise, "ok", label="y with noise")
        plt.plot(tt2, res["fitfunc"](tt2), "r-", label="y fit curve", linewidth=2)
        plt.plot([-10,10],[0,0],'--')
        plt.plot([0,0],[-1,1],'--')
        plt.legend(loc="best")

    curve = res["fitfunc"](tt2)
    curve = (curve-np.min(curve))/(np.max(curve)-np.min(curve))*2-1
    
    phase_fit = res['phase']
    #print ("curve: ", curve.shape, curve[curve.shape[0]//2])

    # COMPUTE PHASE ANGLE FROM AMPLITUDE AND GRADIENT OF SINUSOID
    angle = np.arcsin(curve[curve.shape[0]//2])
    if angle<0:
        angle+=np.pi

    #
    curve0 = curve[curve.shape[0]//2]
    curve1 = curve[curve.shape[0]//2+1]
    if curve0<0:
        if (curve1-curve0)>0:
            angle = np.pi+angle
        else:
            angle = 2*np.pi-angle
    else:
        if (curve1-curve0)<=0:
            angle = np.pi-angle
        else:
            angle = angle

    
    angles_fitted.append(angle)
    
    print ("real angle: "+str(round(phase,4)) 
           + "  curve0: "+ str(round(curve0,4)) 
           + "  computed angle " + str(round(angle,4)))

# plt.title("real angle: "+str(phase) + "  curve0: " + str(curve0)
#           + " Computed angle " + str(angle),fontsize=10)

plt.plot(np.array(angles_fitted)+.1)
plt.plot(angles_truth)
plt.show()

                

real angle: 0.0  curve0: 0.002  computed angle 0.002
real angle: 0.0635  curve0: 0.0654  computed angle 0.0655
real angle: 0.1269  curve0: 0.1286  computed angle 0.1289
real angle: 0.1904  curve0: 0.1912  computed angle 0.1924
real angle: 0.2539  curve0: 0.2531  computed angle 0.2559
real angle: 0.3173  curve0: 0.3139  computed angle 0.3193
real angle: 0.3808  curve0: 0.3735  computed angle 0.3828
real angle: 0.4443  curve0: 0.4316  computed angle 0.4463
real angle: 0.5077  curve0: 0.4879  computed angle 0.5097
real angle: 0.5712  curve0: 0.5423  computed angle 0.5732
real angle: 0.6347  curve0: 0.5945  computed angle 0.6367
real angle: 0.6981  curve0: 0.6443  computed angle 0.7001
real angle: 0.7616  curve0: 0.6915  computed angle 0.7636
real angle: 0.8251  curve0: 0.7359  computed angle 0.8271
real angle: 0.8885  curve0: 0.7774  computed angle 0.8905
real angle: 0.952  curve0: 0.8157  computed angle 0.954
real angle: 1.0155  curve0: 0.8508  computed angle 1.0175
real angle: 1.0789  c